In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
#
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

import tools
import importlib
importlib.reload(tools)

/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


<module 'tools' from '/home/ec2-user/slack-trading/tools.py'>

### Download data from AWS S3

In [2]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.download_data_from_s3(s3_bucket, local_path)

### CPU Count

In [3]:
cpu_count()

16

### Begin

In [4]:
# dataset = pd.read_csv('../data/all_stock_close.csv')
dataset = pd.read_csv('../data/us_hk/us_hk_clean_10yr_stock_close.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', drop=True, inplace=True)
dataset.head(3)

,0001.HK,0002.HK,0003.HK,0004.HK,0005.HK,0006.HK,0007.HK,0008.HK,0010.HK,0011.HK,...,VFC,VRNT,WDC,WFC,WMT,^DJI,^GSPC,^IXIC,^TNX,^VIX
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-28,56.334431,41.966953,5.800245,7.799201,49.283875,26.950289,0.71,1.890068,40.080761,82.240196,...,16.199192,31.690001,26.569702,23.259796,41.967972,11575.540039,1258.510010,2662.879883,3.481,17.520000
2010-12-29,57.905922,42.365368,5.895123,8.097270,49.750565,27.142986,0.72,1.935749,40.827366,83.661453,...,16.249552,31.540001,26.420523,23.125603,42.233482,11585.379883,1259.780029,2666.929932,3.341,17.280001
2010-12-30,57.620216,42.199368,5.888801,8.229015,49.812798,27.170521,0.76,1.952879,41.063133,83.726082,...,16.201061,31.500000,26.420523,22.976505,42.225681,11569.709961,1257.880005,2662.979980,3.369,17.520000


In [5]:
train_start = '2011-01-01'
train_end = '2011-12-31'
dataset = dataset[train_start:train_end]

In [6]:
prevday_returns = np.log(dataset.shift(1)/dataset.shift(2))
today_returns = np.log(dataset/dataset.shift(1))
#
stocks_prev = prevday_returns.columns.to_list()
stocks_today_draft = today_returns.columns.to_list()
stocks_today = [s for s in stocks_today_draft if '.HK' in s]
#
new_col = ['{}.prev'.format(c) for c in stocks_prev]
prevday_returns.columns = new_col
#
comb_df = pd.concat([prevday_returns, today_returns[stocks_today]], axis=1)
comb_df.shape

(245, 1525)

### Fina the NaN in comb_df

In [7]:
comb_df.dropna(axis=0, how='any', inplace=True)

In [8]:
comb_df.tail(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3900.HK,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK
Date,,,,,,,,,,,,,,,,,,,,,
2011-12-28,0.012451,0.002274,0.007808,0.011281,0.014316,0.003512,-0.029853,0.054347,0.042260,0.007547,...,-0.048374,0.005141,-0.002238,-0.017483,-0.020762,-0.057175,0.000000,-0.013129,-0.007435,0.000000
2011-12-29,0.004829,-0.007599,-0.001112,-0.025573,-0.010084,0.006987,-0.015268,-0.003839,-0.005764,-0.004845,...,-0.014685,0.005115,-0.009009,-0.008854,0.000000,-0.060644,-0.020495,-0.013304,-0.030305,0.000000
2011-12-30,-0.009142,-0.000763,-0.006697,-0.004321,-0.013606,-0.008741,0.000000,0.041437,0.009206,-0.003243,...,-0.002963,0.000000,0.011247,-0.002545,0.000000,0.459543,-0.017911,-0.018019,0.015268,0.013247


In [9]:
comb_df.head(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3900.HK,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-05,0.012648,-0.005488,0.002165,0.001615,0.003118,0.001009,-0.027029,0.008607,0.000955,-0.001539,...,0.006369,0.000000,0.002567,0.002462,-0.007203,-0.028982,0.010612,0.040191,0.000000,-0.008772
2011-01-06,0.010938,0.004706,-0.001083,-0.011347,0.023994,0.001009,-0.013793,-0.014388,0.004764,0.017564,...,0.015748,0.056834,-0.033907,0.000000,0.002407,0.000000,-0.017302,-0.021440,-0.006473,0.008268
2011-01-07,0.021523,0.004684,-0.014168,0.004068,0.005456,-0.002018,0.013793,0.014388,0.006632,-0.000757,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063791


In [10]:
prevday_returns = comb_df[new_col]
today_returns = comb_df[stocks_today]

In [11]:
print('{} {}'.format(len(new_col), len(stocks_today)))

812 713


### Load the regression result to identify the two independent variables

In [12]:
result_df = pd.read_csv('../data/us_hk_2indp_var_10yr_regression_pair.csv', index_col=0)
sorted_df = result_df.sort_values(by='RSquared', ascending=False)
sorted_df.head(10)

,Prev,Today,Params,TValues,RSquared
351330,"['MFC.prev', '2000.HK.prev']",0945.HK,0.715477,20.197506,0.638819
350977,"['MFC.prev', '0489.HK.prev']",0945.HK,0.720356,20.079766,0.632327
350925,"['MFC.prev', '0358.HK.prev']",0945.HK,0.744969,20.260874,0.631332
351353,"['MFC.prev', '2314.HK.prev']",0945.HK,0.732811,19.991738,0.625757
351394,"['MFC.prev', '2883.HK.prev']",0945.HK,0.740578,19.926946,0.623319
351430,"['MFC.prev', '3898.HK.prev']",0945.HK,0.733018,19.840290,0.621626
351255,"['MFC.prev', '1199.HK.prev']",0945.HK,0.732026,19.828496,0.621426
351442,"['MFC.prev', '^HSI.prev']",0945.HK,0.739259,19.777927,0.619753
350893,"['MFC.prev', '0293.HK.prev']",0945.HK,0.723748,19.684821,0.618822
351368,"['MFC.prev', '2357.HK.prev']",0945.HK,0.724993,19.667689,0.618059


In [13]:
sorted_df[sorted_df['RSquared'].isna()]

,Prev,Today,Params,TValues,RSquared
324000,"['0001.HK.prev', '0002.HK.prev']",0865.HK,0.0,NaN,NaN
324001,"['0001.HK.prev', '0003.HK.prev']",0865.HK,0.0,NaN,NaN
324002,"['0001.HK.prev', '0004.HK.prev']",0865.HK,0.0,NaN,NaN
324003,"['0001.HK.prev', '0005.HK.prev']",0865.HK,0.0,NaN,NaN
324004,"['0001.HK.prev', '0006.HK.prev']",0865.HK,0.0,NaN,NaN
...,...,...,...,...,...
416335,"['0001.HK.prev', '^DJI.prev']",1159.HK,0.0,NaN,NaN
416336,"['0001.HK.prev', '^GSPC.prev']",1159.HK,0.0,NaN,NaN
416337,"['0001.HK.prev', '^IXIC.prev']",1159.HK,0.0,NaN,NaN
416338,"['0001.HK.prev', '^TNX.prev']",1159.HK,0.0,NaN,NaN


In [14]:
sorted_df.fillna(0, inplace=True)

##### Find the row grouped by 'Today' with max RSquared within each group

In [15]:
best_prev_sofar_df = sorted_df.loc[sorted_df.groupby('Today')['RSquared'].idxmax()]
best_prev_sofar_df

,Prev,Today,Params,TValues,RSquared
770,"['^GSPC.prev', 'MOS.prev']",0001.HK,0.438441,4.366257,0.293187
1550,"['0868.HK.prev', 'F.prev']",0002.HK,-0.073289,5.016278,0.160396
1708,"['WMT.prev', '0158.HK.prev']",0003.HK,0.275360,4.186803,0.110899
3220,"['TWM.prev', 'TCOM.prev']",0004.HK,-0.209796,5.378061,0.322726
3840,"['^GSPC.prev', '2000.HK.prev']",0005.HK,0.829496,13.772557,0.451501
...,...,...,...,...,...
573884,"['0195.HK.prev', '0876.HK.prev']",3989.HK,0.419873,3.725505,0.084285
575005,"['MOS.prev', 'ADSK.prev']",3993.HK,0.424547,6.495086,0.375327
575748,"['BA.prev', '2389.HK.prev']",3998.HK,0.629645,6.177995,0.160721
576037,"['BA.prev', '0226.HK.prev']",3999.HK,0.573978,7.108943,0.190877


In [16]:
best_prev_sofar_df[best_prev_sofar_df['Today']=='0005.HK']

,Prev,Today,Params,TValues,RSquared
3840,"['^GSPC.prev', '2000.HK.prev']",0005.HK,0.829496,13.772557,0.451501


### Regression

In [17]:
def get_linear_regression(pair, prevday_returns, today_returns):
    catstocks = pair['prev']
    column = pair['today']
    X = prevday_returns[catstocks].copy()
    X = sm.add_constant(X)
    y = today_returns[column]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstocks,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared
    }
    # print(data)
    return data

### Generate pair list

In [18]:
catstocks = new_col
pair_list = []
for column in today_returns.columns:
    best_prev_str = best_prev_sofar_df.loc[best_prev_sofar_df['Today']==column, 'Prev'].values[0]
    best_prev_list = best_prev_str.replace('\'','').strip('][').split(', ') # convert list representation to list
    for catstock in catstocks:
        if (column not in catstock) and (catstock not in best_prev_list):
            data = {
                'prev' : best_prev_list + [catstock],
                'today' : column
            }
            pair_list.append(data)
#

In [19]:
pair_list[:5]

[{'prev': ['^GSPC.prev', 'MOS.prev', '0002.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^GSPC.prev', 'MOS.prev', '0003.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^GSPC.prev', 'MOS.prev', '0004.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^GSPC.prev', 'MOS.prev', '0005.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^GSPC.prev', 'MOS.prev', '0006.HK.prev'], 'today': '0001.HK'}]

### NON-Parallel processing

In [ ]:
%%time
results = [get_linear_regression(pair, prevday_returns, today_returns) for pair in pair_list]
result_df = pd.DataFrame(results)
result_df

### Parrallel processing

In [20]:
%%time
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(get_linear_regression)(pair, prevday_returns, today_returns) for pair in pair_list)
results = executor(tasks)
result_df = pd.DataFrame(results)
result_df

cpu count = 16


/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwar

/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1636: RuntimeWarning: inva

CPU times: user 2min 24s, sys: 6.2 s, total: 2min 30s
Wall time: 26min 21s


,Prev,Today,Params,TValues,RSquared
0,"[^GSPC.prev, MOS.prev, 0002.HK.prev]",0001.HK,0.435440,4.345657,0.299432
1,"[^GSPC.prev, MOS.prev, 0003.HK.prev]",0001.HK,0.437923,4.357184,0.294897
2,"[^GSPC.prev, MOS.prev, 0004.HK.prev]",0001.HK,0.442158,4.403472,0.296948
3,"[^GSPC.prev, MOS.prev, 0005.HK.prev]",0001.HK,0.434703,4.347046,0.302267
4,"[^GSPC.prev, MOS.prev, 0006.HK.prev]",0001.HK,0.440134,4.369345,0.293511
...,...,...,...,...,...
576812,"[0237.HK.prev, VFC.prev, ^DJI.prev]",6823.HK,4.054968,9.309450,0.297975
576813,"[0237.HK.prev, VFC.prev, ^GSPC.prev]",6823.HK,4.053944,9.306390,0.297898
576814,"[0237.HK.prev, VFC.prev, ^IXIC.prev]",6823.HK,4.049844,9.287362,0.297818
576815,"[0237.HK.prev, VFC.prev, ^TNX.prev]",6823.HK,4.052983,9.305953,0.298198


In [21]:
result_df.to_csv('../data/us_hk/us_hk_3indp_var_10yr_regression_pair.csv')

### Read Regression Result

In [ ]:
result2_df = pd.read_csv('../data/us_hk/us_hk_3indp_var_10yr_regression_pair.csv', index_col=0)

In [ ]:
sorted2_df = result2_df.sort_values(by='RSquared', ascending=False)
sorted2_df.head(10)

In [ ]:
result_df = pd.read_csv('../data/us_hk/us_hk_2indp_var_10yr_regression_pair.csv', index_col=0)
sorted_df = result_df.sort_values(by='RSquared', ascending=False)
sorted_df.head(10)

### Upload to AWS S3

In [ ]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.upload_data_to_s3(s3_bucket, local_path)

### Verify Linear regression

    catstock = pair['prev']
    column = pair['today']
    X = prevday_returns[[catstock]].copy()
    X = sm.add_constant(X)
    y = today_returns[[column]]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstock,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared
    }

In [ ]:
stock1 = ['0266.HK.prev', '0916.HK.prev']
stock2 = '0952.HK'

In [ ]:
prevday_returns[stock1]

In [ ]:
today_returns[stock2]

In [ ]:
X = prevday_returns[stock1].copy()
X = sm.add_constant(X)
y = today_returns[stock2]
model = sm.OLS(y, X).fit()
print('Params {:.4f} TValues {:.4f} RSquared {:.4f}'.format(model.params.iloc[1], model.tvalues.iloc[1], model.rsquared))